# Import libraries

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/ROBT613_BMI_Final_Project

/content/drive/MyDrive/ROBT613_BMI_Final_Project


In [3]:
!pip install mne==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313874 sha256=4b7caf646936203e69bced8521ff01cef1af422ac322697bd797a0b0d6926ada
  Stored in directory: /root/.cache/pip/wheels/50/55/a4/d95e3c6ab9370b77f9c871e9882a9578a2195c64ede88694ff
Successfully built mne


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import itertools
import pandas as pd
import pickle
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler

from nu_smrutils import loaddat,subject_specific,augment_dataset,crop_data,train_model

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

# Select device

In [5]:
#by default
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

Your GPU device name : Tesla T4


# Select Dataset

## 1. BNCI2014004

In [6]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


# Load subject specific data, preprocess data (filtering, data augmentation, data cropping, standard scaling), and use Pytorch dataloaders

Function for standard scaler

In [7]:
scaler = StandardScaler()
def standard_scaling(data):
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  data = data.reshape(data.shape[0],-1)
  data = scaler.fit_transform(data)
  data = data.reshape(data.shape[0],shape1,shape2)
  return data

Function for Pytorch dataloaders

In [8]:
from torch.utils.data import TensorDataset, DataLoader

def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):
    # convert data dimensions to into to gray scale image format
    #if EEGNET: ### EEGNet model requires the last dimension to be 1
        #ff = lambda dat: torch.unsqueeze(dat, dim = -1)
    #else:
        #ff = lambda dat: torch.unsqueeze(dat, dim = 1)

    x_train, x_valid, x_test =  (dat['xtrain'],dat['xvalid'],dat['xtest'])
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)

    # TensorDataset & Dataloader
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev))
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev))

    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader},
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})
    return output

In [9]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Data cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
to

# Transformer architecture




In [10]:
class EEGTransformer(nn.Module):
    def __init__(self, num_channels, num_timepoints, output_dim,
                 hidden_dim, num_heads, key_query_dim,
                 hidden_ffn_dim, intermediate_dim, ffn_output_dim):
        super(EEGTransformer, self).__init__()

        # Positional Encoding
        self.positional_encoding = torch.zeros(num_channels, num_timepoints)
        for j in range(num_channels):
            for k in range(num_timepoints):
                if j % 2 == 0:
                    self.positional_encoding[j][k] =\
                        torch.sin(torch.tensor(k) / (10000 ** (torch.tensor(j) / num_channels)))
                else:
                    self.positional_encoding[j][k] =\
                        torch.cos(torch.tensor(k) / (10000 ** ((torch.tensor(j) - 1) / num_channels)))

        # Multi-Head Self Attention
        self.multihead_attn = nn.MultiheadAttention(embed_dim=num_channels,
                                                    num_heads=num_heads)

        # Feed-Forward Network
        self.ffn = nn.Sequential(
            nn.Linear(num_channels, intermediate_dim),
            nn.ReLU(),
            nn.Linear(intermediate_dim, ffn_output_dim)
        )

        # Layer Normalization
        self.norm1 = nn.LayerNorm(num_channels)
        self.norm2 = nn.LayerNorm(num_channels)

        # Classifier
        self.classifier = nn.Linear(num_channels * num_timepoints, output_dim)

    def forward(self, X):
        # Input Standardization
        mean = X.mean(dim=2, keepdim=True)
        std  = X.std(dim=2, keepdim=True)
        X_hat = (X - mean) / (std + 1e-5)  # epsilon to avoid division by zero

        # Add Positional Encoding
        X_tilde = X_hat + self.positional_encoding.to(X.device)

        # Reshape for multi-head self attention: (seq_len, batch_size, embed_dim)
        X_tilde = X_tilde.permute(2, 0, 1)

        # Multi-Head Self Attention
        attn_output, _ = self.multihead_attn(X_tilde, X_tilde, X_tilde)

        # Reshape back and Apply Layer Norm
        # attn_output = attn_output.permute(1, 2, 0)  # Reshape: (batch_size, embed_dim, seq_len)
        X_ring = torch.stack([self.norm1(a) for a in attn_output], dim=1)

        # Position-wise Feed-Forward Networks
        ff_output = self.ffn(X_ring)
        O = self.norm2(ff_output + X_ring)

        # Classifier
        # Flatten and classify
        O_flat = O.view(O.size(0), -1)  # Flatten the tensor
        output = self.classifier(O_flat)

        return output

# Training loop

In [13]:
#Hyperparameter settings
num_epochs = 200
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = EEGTransformer(input_size[1], input_size[2], output_dim=2,
                       hidden_dim=512, num_heads=1, key_query_dim=512,
                       hidden_ffn_dim=512, intermediate_dim=2048,
                       ffn_output_dim=input_size[1])


  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)


  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 157/200
train loss: 0.0105, acc: 0.5878
val loss: 0.0127, acc: 0.5362
Epoch 158/200
train loss: 0.0105, acc: 0.5929
val loss: 0.0127, acc: 0.5217
Epoch 159/200
train loss: 0.0105, acc: 0.5958
val loss: 0.0127, acc: 0.5507
Epoch 160/200
train loss: 0.0105, acc: 0.5881
val loss: 0.0126, acc: 0.5181
Epoch 161/200
train loss: 0.0105, acc: 0.5894
val loss: 0.0127, acc: 0.5254
Epoch 162/200
train loss: 0.0105, acc: 0.6000
val loss: 0.0127, acc: 0.5399
Epoch 163/200
train loss: 0.0105, acc: 0.5913
val loss: 0.0127, acc: 0.5036
Epoch 164/200
train loss: 0.0105, acc: 0.5933
val loss: 0.0127, acc: 0.5217
Epoch 165/200
train loss: 0.0105, acc: 0.5872
val loss: 0.0127, acc: 0.5399
Epoch 166/200
train loss: 0.0105, acc: 0.5888
val loss: 0.0127, acc: 0.5254
Epoch 167/200
train loss: 0.0105, acc: 0.5875
val loss: 0.0127, acc: 0.5326
Epoch 168/200
train loss: 0.0105, acc: 0.5811
val loss: 0.0127, acc: 0.5362
Epoch 169/200
train 

# Save and print results

In [14]:
table.to_csv('results/Transformer/BNCI2014004_transformer.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.532051  0.557971      0.34     11
Subject2   0.549559  0.555556      0.62     48
Subject3   0.500962  0.539855      0.48      1
Subject4   0.611735  0.533333      0.50    102
Subject5   0.508739  0.564912      0.54      3
Subject6   0.584936  0.532609      0.50     52
Subject7   0.630128  0.623188      0.54    156
Subject8   0.642987  0.594502      0.56    197
Subject9   0.612179  0.554348      0.56     82

Average between subjects: 0.52
Max test_score between subjects: 0.62


Repeat the same steps for other datasets
## 2. BNCI2014001

In [15]:
itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


In [16]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.000001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Data cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

In [17]:
#Hyperparameter settings
num_epochs = 200
learning_rate = 1e-3
weight_decay = 1e-3
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = EEGTransformer(input_size[1], input_size[2], output_dim=2,
                       hidden_dim=512, num_heads=2, key_query_dim=512,
                       hidden_ffn_dim=512, intermediate_dim=2048,
                       ffn_output_dim=input_size[1])

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 158/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0354, acc: 0.5946
Epoch 159/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0350, acc: 0.5856
Epoch 160/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0354, acc: 0.6036
Epoch 161/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0352, acc: 0.5946
Epoch 162/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0351, acc: 0.5946
Epoch 163/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0352, acc: 0.5946
Epoch 164/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0350, acc: 0.5946
Epoch 165/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0349, acc: 0.5946
Epoch 166/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0350, acc: 0.5856
Epoch 167/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0351, acc: 0.5766
Epoch 168/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0344, acc: 0.5856
Epoch 169/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0351, acc: 0.5856
Epoch 170/200
train 

In [18]:
table.to_csv('results/Transformer/BNCI2014001_transformer.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.882448  0.675676      0.68     11
Subject2   0.946860  0.612613      0.52     22
Subject3   0.983092  0.693694      0.58     84
Subject4   0.992754  0.567568      0.48     15
Subject5   0.579710  0.549550      0.50      2
Subject6   1.000000  0.549550      0.56    168
Subject7   0.815620  0.639640      0.52     11
Subject8   0.891304  0.819820      0.82      6
Subject9   1.000000  0.810811      0.76     61

Average between subjects: 0.60
Max test_score between subjects: 0.82


## 3. Weibo2014

In [19]:
itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


In [20]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Data cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
tor

In [21]:
#Hyperparameter settings
num_epochs = 200
learning_rate = 1e-3
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = EEGTransformer(input_size[1], input_size[2], output_dim=2,
                       hidden_dim=512, num_heads=4, key_query_dim=512,
                       hidden_ffn_dim=512, intermediate_dim=2048,
                       ffn_output_dim=input_size[1])

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 158/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0518, acc: 0.4762
Epoch 159/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0530, acc: 0.4921
Epoch 160/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0515, acc: 0.4762
Epoch 161/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0502, acc: 0.5238
Epoch 162/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0513, acc: 0.5079
Epoch 163/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0536, acc: 0.4762
Epoch 164/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0525, acc: 0.4762
Epoch 165/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0520, acc: 0.4921
Epoch 166/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0509, acc: 0.4921
Epoch 167/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0502, acc: 0.5238
Epoch 168/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0547, acc: 0.4921
Epoch 169/200
train loss: 0.0000, acc: 1.0000
val loss: 0.0530, acc: 0.4921
Epoch 170/200
train 

In [22]:
table.to_csv('results/Transformer/Weibo2014_transformer.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1    0.650725  0.619048      0.46      4
Subject2    0.588406  0.619048      0.60      3
Subject3    0.847826  0.650794      0.50     10
Subject4    0.937681  0.650794      0.50     11
Subject5    0.791304  0.587302      0.48      9
Subject6    1.000000  0.666667      0.56     89
Subject7    1.000000  0.603175      0.62     49
Subject8    0.779710  0.571429      0.50      7
Subject9    1.000000  0.587302      0.66     35
Subject10   0.575362  0.523810      0.64      2

Average between subjects: 0.55
Max test_score between subjects: 0.66
